In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

In [ ]:
# Constant values

DATA_DIR = '/content/dataset'
CATEGORIES = ['COVID', 'non-COVID']

IMG_SIZE = 200

In [ ]:
training_data = []

# This function creates normalized data for training
def create_training_data():
  for category in CATEGORIES:
    path = os.path.join(DATA_DIR, category)
    class_num = CATEGORIES.index(category)

    for image in os.listdir(path):
      gray_arr = cv2.imread(os.path.join(path, image), cv2.IMREAD_GRAYSCALE)
      resized_arr = cv2.resize(gray_arr, (IMG_SIZE, IMG_SIZE))

      training_data.append([resized_arr, class_num])

create_training_data()

In [ ]:
# Count of all images
len(training_data)

2481

In [ ]:
# Shuffling training data for better training process
import random

random.shuffle(training_data)

In [ ]:
for data in training_data[:10]:
  print(data[1])

0
0
0
1
1
1
0
1
1
1


In [ ]:
# image data and label (covid or not) variables
X = []
Y = []

In [ ]:
# splitting image data and label to variables
for data, label in training_data:
  X.append(data)
  Y.append(label)

# converting lists to numpy arrays for tf
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
Y = np.array(Y)

In [ ]:
# saving np arrays for later use
np.save('/content/drive/MyDrive/data.npy', X)
np.save('/content/drive/MyDrive/labels.npy', Y)

In [ ]:
# loading np arrays again
X = np.load('/content/drive/MyDrive/data.npy')
Y = np.load('/content/drive/MyDrive/labels.npy')

In [ ]:
# All training process

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D

model = Sequential()

model.add(Conv2D(64, (2,2), input_shape = X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D())

model.add(Conv2D(64, (2,2)))
model.add(Activation('relu'))
model.add(MaxPooling2D())

model.add(Conv2D(64, (2,2)))
model.add(Activation('relu'))
model.add(MaxPooling2D())

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(128, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
# You can load this pre-trained model
# model = tf.keras.models.load_model('/content/drive/MyDrive/covid-ct-cnn-64x3-512-1614076563.h5')

model.fit(X, Y, epochs=25, validation_split=0.1)

In [ ]:
# Saving model for later use
import time

model.save('/content/drive/MyDrive/covid-ct-cnn-64x3-512-last-{}.h5'.format(int(time.time())))

# **Predict Time!**

In [ ]:
import cv2
import tensorflow as tf

# for get rid of warnings set logger level to errors
tf.get_logger().setLevel('ERROR')

CATEGORIES = ['COVID', 'non-COVID']

# preparing input images to as our model can understand
def prepare(path):
  IMG_SIZE = 200
  img_arr = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
  resized_arr = cv2.resize(img_arr, (IMG_SIZE, IMG_SIZE))
  return resized_arr.reshape(-1, IMG_SIZE, IMG_SIZE, 1)

# loading model
model = tf.keras.models.load_model('/content/drive/MyDrive/models/covid-ct-cnn-64x3-512-last-1614079489.h5')

# get input
prediction = model.predict([prepare('/content/images.jfif')])

# prediction output
print(CATEGORIES[int(prediction[0][0])])


COVID
